## Variational Quantum Eigensolver


**Goal:** Find the parameter $\theta$ that minimizes the energy $E(\theta)$ of a state prepared by a parameterized circuit $U(\theta)$.

**Hamiltonian:** $H = Z_0$. Eigenstates are $|0\rangle$ (Energy $E=+1$) and $|1\rangle$ (Energy $E=-1$). The ground state is $|1\rangle$ with energy -1.

**Ansatz Circuit $U(\theta)$:** We'll use a rotation around the Y-axis, $U(\theta)=R_y(\theta)$. Since we don't have a generic $R_y$ operation derived, we can construct it using Hadamard analogs (`Superposition`, $H$) and a Z-rotation (`PhaseShift`, analogous to $R_z$ up to global phase): $R_y(\theta) \approx H \cdot R_z(\theta) \cdot H$.

**ONQ Objective Function:** Prepare state $|\psi(\theta)\rangle = R_y(\theta)|0\rangle$. Perform `Stabilize` on QDU 0. Record the outcome $m_0 \in \{0, 1\}$. Our energy analog is calculated as $E(\theta) = (-1)^{m_0}$. (If $m_0=0$, $E=+1$; if $m_0=1$, $E=-1$).

**Procedure:** We simulate this process for various $\theta$ values from $0$ to $2\pi$ and plot $E(\theta)$ vs $\theta$. We expect the minimum energy (-1) when $R_y(\theta)|0\rangle$ produces the $|1\rangle$ state, which occurs quantum mechanically at $\theta = \pi$. The ONQ simulation will show the deterministic energy outcome for each $\theta$ based on its stabilization rules.

In [ ]:
// This import style lets us update notebooks without having to push a new cargo package
:dep onq = { version = "0.3.0", path = "../" }
:dep plotters = { version = "^0.3.7", default_features = false, features = ["evcxr", "line_series"] }

In [ ]:
extern crate plotters;
use plotters::prelude::*;

In [ ]:
use onq::{
    core::{OnqError, QduId, StableState},
    operations::Operation,
    vm::{Instruction, Program, ProgramBuilder, OnqVm}, // Use full VM for potential future extension
    // Note: Simulator could also be used if no classical feedback needed *during* the run
};
use std::f64::consts::PI;

In [ ]:
// Helper for QduId creation
fn qid(id: u64) -> QduId { QduId(id) }

// --- VQE Parameters ---
let q0 = qid(0); // The single QDU for our H=Z0 problem
let num_theta_steps: usize = 50; // Number of parameter values to test

println!("Setup complete. QDU: {}, Theta steps: {}", q0, num_theta_steps);

In [ ]:
// --- Simulation Loop ---

let mut results_data: Vec<(f64, f64)> = Vec::with_capacity(num_theta_steps);
let mut vm = OnqVm::new(); // Create VM instance outside the loop

println!("Running VQE analog loop for H = Z_0...");

for i in 0..=num_theta_steps {
    let theta = (i as f64 / num_theta_steps as f64) * 2.0 * PI; // Angle from 0 to 2*PI

    // Build program for this theta value
    // U(theta) = H * Rz(theta) * H applied to |0>
    // Rz(theta) is implemented via PhaseShift(theta) - check definitions carefully.
    // Standard Rz(t) = diag(exp(-it/2), exp(it/2)). PhaseShift(t) = diag(1, exp(it)).
    // Relationship: Rz(t) = exp(-it/2) * PhaseShift(t). They differ by global phase.
    // Since global phase doesn't affect stabilization outcome, using PhaseShift is fine here.
    let program = ProgramBuilder::new()
        // Prepare Ry(theta)|0> state
        .pb_add(Instruction::QuantumOp(Operation::InteractionPattern{ target: q0, pattern_id: "Superposition".to_string() })) // H
        .pb_add(Instruction::QuantumOp(Operation::PhaseShift{ target: q0, theta: theta })) // Rz(theta) analog
        .pb_add(Instruction::QuantumOp(Operation::InteractionPattern{ target: q0, pattern_id: "Superposition".to_string() })) // H
        // Stabilize and record result
        .pb_add(Instruction::Stabilize{ targets: vec![q0] })
        .pb_add(Instruction::Record{ qdu: q0, register: "m0".to_string() })
        .pb_add(Instruction::Halt)
        .build()
        .expect("Failed to build program"); // Use expect for example simplicity

    // Run the VM
    match vm.run(&program) {
        Ok(()) => {
            let m0 = vm.get_classical_register("m0");
            // Calculate Energy E = (-1)^m0 : +1 for |0>, -1 for |1>
            let energy = if m0 == 0 { 1.0 } else { -1.0 };
            results_data.push((theta, energy));
            // Optional: Print progress
            // if i % 5 == 0 { println!("  Theta: {:.3}, Outcome m0: {}, Energy: {:.1}", theta, m0, energy); }
        }
        Err(e) => {
            eprintln!("VM run failed for theta={:.3}: {}", theta, e);
            // Push NaN or skip point if error occurs? Skip for now.
            // results_data.push((theta, f64::NAN));
        }
    }
}

println!("Simulation loop finished. Collected {} data points.", results_data.len());
// Display first few results for verification
// println!("First few results: {:?}", &results_data[0..5.min(results_data.len())]);

In [ ]:
// Define plot dimensions
let figure_width = 600;
let figure_height = 400;

// Use evcxr_figure. Make this the LAST expression in the cell.
// Its return value (an SVGWrapper) will be displayed by evcxr.
evcxr_figure((figure_width, figure_height), |root| {
    // Fill background (optional, but good practice)
    root.fill(&WHITE)?; // Example background color

    // Configure chart
    let mut chart = ChartBuilder::on(&root)
        .caption("ONQ VQE Analog: E(θ) for H=Z₀", ("sans-serif", 20).into_font())
        .margin(10)
        .x_label_area_size(30)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..(2.0 * PI), -1.1f64..1.1f64)?; // X: theta, Y: Energy

    // Configure mesh and labels
    chart.configure_mesh()
        .x_desc("Parameter θ (radians)")
        .y_desc("Energy E(θ) = (-1)^m₀")
        // Format x-axis labels as multiples of PI
        .x_labels(5)
        .x_label_formatter(&|x| format!("{:.1}π", x / PI))
        .draw()?;

    // Plot the data series
    chart.draw_series(LineSeries::new(
        // Ensure results_data is accessible here (defined in previous cell)
        results_data.iter().map(|(theta, energy)| (*theta, *energy)), // Map data points
        &BLUE, // Plot color
    ))?;

    // Add legend (optional)
    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperRight)
        .border_style(&BLACK)
        .background_style(&WHITE.mix(0.8))
        .draw()?;

    println!("Plot generated successfully."); // Console message appears above plot
    root.present()?; // Ensure drawing area is finalized
    Ok(()) // Return Ok result from closure
}).style("width:80%") // Optional: Adjust display width in Jupyter output cell